## Build smdebug and update Dockerfile

In [ ]:
! mkdir docker/pip_packages
! cd ../../../../; python setup.py bdist_wheel; mv dist/* smdebug/profiler/analysis/rules/docker/pip_packages/; cd smdebug/profiler/analysis/rules/;
! cd docker; ls pip_packages/smdebug* > requirements.txt; cd ..;


## Build rule container and upload it to Amazon ECR

In [ ]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-profiler-rules-container'
tag = ':latest'

region = boto3.session.Session().region_name

uri_suffix = 'amazonaws.com'
processing_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository docker
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

## Run SageMaker training job with Profiler

In [ ]:
hyperparameters = {'epoch': 10, 
                   'batch_size': 64,
                   'data_augmentation': True}

In [ ]:
import boto3

session = boto3.session.Session()
region = session.region_name

image_name = f'385479125792.dkr.ecr.{region}.amazonaws.com/profiler-gpu:latest'
print(f"image being used is {image_name}")

In [ ]:
import sagemaker
role = sagemaker.get_execution_role()

import boto3
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker.profiler import ProfilerConfig 

estimator = TensorFlow(
    role=sagemaker.get_execution_role(),
    image_name=image_name,
    train_instance_count=1,
    train_instance_type='ml.p3.8xlarge',
    entry_point='train_tf.py',
    source_dir='../../../../examples/profiler/demo',
    framework_version='2.2.0',
    py_version='py37',
    profiler_config=ProfilerConfig(profiling_interval_millis=500),
    script_mode=True,
    hyperparameters=hyperparameters
)

In [ ]:
estimator.fit(wait=False)

## Run builtin rules as SageMaker Processing job

In [ ]:
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput

processor = Processor(
            role=role,
            image_uri=processing_repository_uri,
            instance_count=1,
            instance_type='ml.t3.medium',
            env={'S3_PATH': estimator.latest_job_profiler_artifacts_path()}
        )


In [ ]:
processor.run([], 
              [ProcessingOutput(output_name='profiler-analysis', 
                                source='/opt/ml/processing/outputs', 
                                destination=estimator.latest_job_profiler_artifacts_path())])